In [53]:
import warnings
import bootcamp_utils

In [54]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


2.1

In [ ]:
with open('/Users/alexanderwang/git/bootcamp/data/salmonella_spi1_region.fna', 'r') as f:
    print(f.readlines())

In [23]:
def read_fasta(file_name):
    with open(file_name, 'r') as f:
        lines = f.readlines()
        desc, seq = lines[0].rstrip(), ''.join([s.rstrip() for s in lines[1:]])
    return desc, seq

In [40]:
salmonella_desc, salmonella_seq = read_fasta('/Users/alexanderwang/git/bootcamp/data/salmonella_spi1_region.fna')

2.2

In [5]:
lambda_desc, lambda_seq = read_fasta('lambdafsa.txt')

In [19]:
def restriction_sites(seq, recog_seq):
    sites = []
    prev = seq.find(recog_seq)
    while prev != -1:
        sites.append(prev)
        prev = seq.find(recog_seq, prev + 1)
    return sites

In [22]:
print('AAGCTT: ' + str(restriction_sites(lambda_seq, 'AAGCTT')))
print('GAATTC: ' + str(restriction_sites(lambda_seq, 'GAATTC')))
print('GGTACC: ' + str(restriction_sites(lambda_seq, 'GGTACC')))

AAGCTT: [23129, 25156, 27478, 36894, 37458, 37583, 44140]
GAATTC: [21225, 26103, 31746, 39167, 44971]
GGTACC: [17052, 18555]


In [67]:
len(lambda_seq)

48502

2.3

In [31]:
def gc_content(seq):
    return (seq.count('G') + seq.count('g') + seq.count('C') + seq.count('c')) / len(seq)

def gc_blocks(seq, block_size):
    gcs = []
    while len(seq) > block_size:
        gcs.append(gc_content(seq[:block_size]))
        seq = seq[block_size:]
    return tuple(gcs)

In [32]:
gc_blocks(seq = 'ATGACTACGT', block_size = 4)

(0.25, 0.5)

In [38]:
def gc_map(seq, block_size, gc_thresh):
    gcb = gc_blocks(seq, block_size)
    seq_acc, it = '', 0
    for b in gcb:
        if b >= gc_thresh:
            seq_acc += seq[it:it + block_size].upper()
        else:
            seq_acc += seq[it:it + block_size].lower()
        it += block_size
    return seq_acc

In [39]:
gc_map('ATGACTACGT', 4, 0.4)

'atgaCTAC'

In [41]:
salmonella_gc_map = gc_map(seq = salmonella_seq, block_size = 1000, gc_thresh = 0.45)

In [ ]:
salmonella_gc_map

In [45]:
LINE_LENGTH = 60
temp_seq = salmonella_gc_map
with open('salmonella_gc_map.fasta','w') as out:
    out.write(salmonella_desc + '\n')
    while len(temp_seq) > LINE_LENGTH:
        out.write(temp_seq[:LINE_LENGTH] + '\n')
        temp_seq = temp_seq[LINE_LENGTH:]
    out.write(temp_seq)

2.4

In [80]:
def longest_orf(seq):
    start_codon = 'ATG'
    stop_codons = ['TGA', 'TAG', 'TAA']
    cur_atg, cur_max_start, cur_max_length, it = -1, 0, 0, 0
    while it < len(seq) - 2:
        if cur_atg == -1 and seq[it:it + 3] == start_codon:
            cur_atg = it
        if cur_atg != -1 and seq[it:it + 3] in stop_codons:
            if it + 3 - cur_atg > cur_max_length:
                cur_max_start, cur_max_length = cur_atg, it + 3 - cur_atg
            cur_atg = -1
        if cur_atg != -1:
            it += 2
        it += 1
    return seq[cur_max_start : cur_max_start + cur_max_length - 3]

In [48]:
longest_orf('GGATGATGATGTAAAAC')

'ATGATGATGTAA'

In [51]:
salmonella_orf = longest_orf(salmonella_seq)
salmonella_orf

'ATGACCAACTACAGCCTGCGCGCACGCATGATGATTCTGATCCTGGCCCCGACCGTCCTGATAGGTTTGCTGCTCAGTATCTTTTTTGTAGTGCACCGCTATAACGACCTGCAGCGTCAACTGGAAGATGCCGGCGCCAGTATTATTGAACCGCTCGCCGTCTCCAGCGAATATGGTATGAACTTACAAAACCGGGAGTCTATCGGCCAACTTATCAGCGTCCTGCACCGCAGACACTCTGATATTGTGCGGGCGATTTCCGTTTATGACGATCATAACCGTCTGTTTGTAACGTCTAATTTCCATCTGGACCCCTCACAGATGCAGCTTCCCGCCGGAGCGCCGTTTCCACGTCGTCTGAGCGTTGATCGCCACGGCGATATTATGATTCTGCGCACCCCAATTATCTCGGAGAGCTATTCGCCGGACGAGTCAGCGATTGCTGACGCGAAAAATACCAAAAATATGCTGGGGTATGTGGCGCTTGAACTGGATCTCAAGTCGGTCAGGCTACAGCAATACAAAGAGATTTTTATCTCCAGCGTGATGATGCTTTTTTGTATTGGCATTGCGCTGATCTTTGGCTGGCGGCTTATGCGCGATGTCACCGGGCCTATCCGTAATATGGTGAATACCGTTGACCGCATTCGCCGCGGACAACTGGATAGCCGGGTGGAAGGATTTATGCTGGGCGAACTGGATATGCTGAAAAACGGCATTAATTCCATGGCGATGTCGCTTGCCGCCTATCACGAAGAGATGCAGCATAATATCGATCAGGCCACTTCGGACCTGCGTGAAACCCTTGAGCAGATGGAAATCCAAAACGTTGAGCTGGATCTGGCGAAAAAGCGTGCCCAGGAAGCGGCGCGTATTAAGTCGGAGTTCCTGGCGAACATGTCGCACGAACTGCGAACGCCGCTGAACGGCGTCATTGGCTTTACCCGCCTGACATTAAAAACGGAGCTGAATCCCACCCAGCGCGACCATCTGAACACC

In [56]:
bootcamp_utils.dna_to_rna(longest_orf('GGATGATGATGTAAAAC'))

'AUGAUGAUGUAA'

In [62]:
def seq_to_protein(seq):
    it = 0
    protein_acc = ''
    while it < len(seq) - 2:
        protein_acc += bootcamp_utils.codons[seq[it:it+3]]
        it += 3
    return protein_acc

In [65]:
longest_orf('GGATGATGATGTAAAAC')

'ATGATGATGTAA'

In [63]:
seq_to_protein(longest_orf('GGATGATGATGTAAAAC'))

'MMM*'

In [66]:
seq_to_protein(salmonella_orf)

'MTNYSLRARMMILILAPTVLIGLLLSIFFVVHRYNDLQRQLEDAGASIIEPLAVSSEYGMNLQNRESIGQLISVLHRRHSDIVRAISVYDDHNRLFVTSNFHLDPSQMQLPAGAPFPRRLSVDRHGDIMILRTPIISESYSPDESAIADAKNTKNMLGYVALELDLKSVRLQQYKEIFISSVMMLFCIGIALIFGWRLMRDVTGPIRNMVNTVDRIRRGQLDSRVEGFMLGELDMLKNGINSMAMSLAAYHEEMQHNIDQATSDLRETLEQMEIQNVELDLAKKRAQEAARIKSEFLANMSHELRTPLNGVIGFTRLTLKTELNPTQRDHLNTIERSANNLLAIINDVLDFSKLEAGKLILESIPFPLRNTLDEVVTLLAHSSHDKGLELTLNIKNDVPDNVIGDPLRLQQVITNLVGNAIKFTESGNIDILVEKRALSNTKVQIEVQIRDTGIGIPERDQSRLFQAFRQADASISRRHGGTGLGLVITQKLVNEMGGDISFHSQPNRGSTFWFHINLDLNPNVIIDGPSTACLAGKRLAYVEPNATAAQCTLDLLSDTPVEVVYSPTFSALPLAHYDIMILSVPVTFREPLTMQHERLAKAASMTDFLLLALPCHAQINAEKLKQGGAAACLLKPLTSTRLLPALTEYCQLNHHPEPLLMDTSKITMTVMAVDDNPANLKLIGALLEDKVQHVELCDSGHQAVDRAKQMQFDLILMDIQMPDMDGIRACELIHQLPHQQQTPVIAVTAHAMAGQKEKLLSAGMNDYLAKPIEEEKLHNLLLRYKPGANVAARLMAPEPAEFIFNPNATLDWQLALRQAAGKPDLARDMLQMLIDFLPEVRNKIEEQLVGENPNGLVDLVHKLHGSCGYSGVPRMKNLCQLIEQQLRSGVHEEELEPEFLELLDEMDNVAREAKKILG*'

In [81]:
def n_longest_orf(seq, n=5):
    start_codon = 'ATG'
    stop_codons = ['TGA', 'TAG', 'TAA']
    cur_atg, cur_maxes, it = -1, [], 0
    for i in range(n):
        cur_maxes.append((0,0))
    while it < len(seq) - 2:
        if cur_atg == -1 and seq[it:it + 3] == start_codon:
            cur_atg = it
        if cur_atg != -1 and seq[it:it + 3] in stop_codons:
            if it + 3 - cur_atg > min(cur_maxes[0]):
                cur_maxes.remove(min(cur_maxes))
                cur_maxes.append((it + 3 - cur_atg, cur_atg))
            cur_atg = -1
        if cur_atg != -1:
            it += 2
        it += 1
    seqs = []
    for size, s in cur_maxes:
        seqs.append(seq[s : s + size - 3])
    return seqs

In [82]:
for s in n_longest_orf(salmonella_seq):
    print(seq_to_protein(s))
    print()

MSYTPMSDLGQQGLFDITRTLLQQPDLASLSEALSQLVKRSALADSAGIVLWQAQSQRAQYYATRENGRPVEYEDETVLAHGPVRRILSRPDALHCNFHEFTETWPQLAASGLYPEFGHYCLLPLAAEGRIFGGCEFIRQEDRPWSEKEYDRLHTFTQIVGVVAEQIQNRVNNNVDYDLLCRERDNFRILVAITNAVLSRLDIDELVSEVAKEIHHYFNIDAISIVLRSHRKNKLNIYSTHYLDEHHPAHEQSEVDEAGTLTERVFKSKEMLLINLNERDPLAPYERMLFDTWGNQIQTLCLLPLMSGKTMLGVLKLAQCEEKVFTTANLKLLRQIAERVAIAVDNALAYQEIHRLKERLVDENLALTEQLNNVDSEFGEIIGRSEAMYNVLKQVEMVAQSDSTVLILGETGTGKELIARAIHNLSGRSGRRMVKMNCAAMPAGLLESDLFGHERGAFTGASAQRIGRFELADKSSLFLDEVGDMPLELQPKLLRVLQEQEFERLGSNKLIQTDVRLIAATNRDLKKMVADREFRNDLYYRLNVFPIQLPPLRERPEDIPLLVKAFTFKIARRMGRNIDSIPAETLRTLSSMEWPGNVRELENVVERAVLLTRGNVLQLSLPDITAVTPDTSPVATESAKEGEDEYQLIIRVLKETNGVVAGPKGAAQRLGLKRTTLLSRMKRLGIDKDALA

MPHFNPVPVSNKKFVFDDFILNMDGSLLRSEKKVNIPPKEYAVLVILLEAAGEIVSKNTLLDQVWGDAEVNEESLTRCIYALRRILSEDKEHRYIETLYGQGYRFNRPVVVVSPPAPQPTTHTLAILPFQMQDQVQSESLHYSIVKGLSQYAPFGLSVLPVTITKNCRSVKDILELMDQLRPDYYISGQMIPDGNDNIVQIEIVRVKGYHLLHQESIKLIEHQPASLLQNKIANLLLRCIPGLRWDTKQISELNSIDSTMVYLRGKHELNQYTPYSLQQALKLLTQCVNMSPNSIAPYCALAECYL